# Indexing
- **Task**: index tokens in documents and weigth their relevance.
- **Input**: tokenized documents
- **Output**: term-document matrix

## Main steps
1. Tf (Term frequency)
2. Idf (Inverse document frequency)
3. TfIdf

In [2]:
import json
import pandas as pd
import numpy as np
from collections import Counter, defaultdict
import spacy
import nltk

In [3]:
dataset_file = '../data/wiki_dataset.json'
with open(dataset_file, 'r') as infile:
    dataset = json.load(infile)
docs = dataset['docs']
queries = dataset['queries']

In [4]:
nlp = spacy.load("en_core_web_sm")

In [7]:
tokens = lambda text: [x.lemma_ for x in nlp(text) if x.pos_ not in ['PUNCT', 'SPACE'] and not x.is_stop]
all_tokens = lambda text: [x.lemma_ for x in nlp(text) if x.pos_ not in ['PUNCT', 'SPACE']]

## Indexing from scratch
Index structure: <code>{doc_id: {token: tf(token, doc_id), ...}, ...}</code>

### Tf
In this example, we exploit double normalized K tf, with K = 0.5:
$$
tf(t, d) = K + (1 - K)\frac{f(t,d)}{\max\limits_{t' \in d} f(t',d)},
$$
where $f(t,d)$ is the frequency (i.e., count) of token $t$ in document $d$.

In [8]:
TF, k = {}, 0.5
for docid, text in enumerate(docs):
    f = Counter(all_tokens(text)).most_common()
    maxf = f[0][1]
    TF[docid] = dict([(token, k + (1 - k) * (x / maxf)) for token, x in f])

In [9]:
docs[124]

'Sabine Bramhoff (born 1 November 1964) is a retired German high jumper. She finished seventh at the 1989 European Indoor Championships. She represented the sports club LC Paderborn, and won the silver medal at the West German champion in 1989. Her personal best jump was 1.94 metres (6.3\xa0ft), achieved in August 1990 in Düsseldorf.'

#### Bag of words

In [10]:
list(sorted(TF[124].items(), key=lambda x: -x[1]))[:10]

[('the', 1.0),
 ('-PRON-', 0.875),
 ('in', 0.875),
 ('be', 0.75),
 ('german', 0.75),
 ('at', 0.75),
 ('1989', 0.75),
 ('Sabine', 0.625),
 ('Bramhoff', 0.625),
 ('bear', 0.625)]

### Idf
$$
idf(t) = \log \left(\frac{N}{n_t} \right),
$$
where $N$ denotes the corpus size, and $n_t$ denotes the number of documents actually containing $t$.

In [11]:
DF, N = defaultdict(lambda: 0), len(docs)
for k, bow in TF.items():
    for t in bow.keys():
        DF[t] += 1
IDF = lambda x: np.log(N / DF[x])

In [13]:
print(DF['the'], DF['chinese'])
print(IDF('the'), IDF('chinese'))

2518 8
0.21692888706090915 5.968707559985366


### TfIdf

In [14]:
TfIdf = {}
for k, bow in TF.items():
    TfIdf[k] = dict([(token, w * IDF(token)) for token, w in bow.items()])

In [15]:
list(sorted(TfIdf[124].items(), key=lambda x: -x[1]))[:10]

[('Bramhoff', 5.030093188540751),
 ('LC', 5.030093188540751),
 ('Paderborn', 5.030093188540751),
 ('1.94', 5.030093188540751),
 ('ft', 5.030093188540751),
 ('jumper', 4.596876200690786),
 ('Indoor', 4.596876200690786),
 ('6.3', 4.596876200690786),
 ('Düsseldorf', 4.596876200690786),
 ('Sabine', 4.16365921284082)]

## Multiword indexing and compound terms selection
**Task**: find sequences of n words (called n_grams) that should be counted as a single word during indexing (i.e., New York).

**Approch**: use pointwise mutual information to estimate the probability of a n_gram (say a 2_gram in the example) to be a single compound term

$$
pmi(t_i, t_j) = \log \left (\frac{P(t_i, t_j)}{P(t_i)P(t_j)} \right ) 
$$

Denoting $f(t)$ the frequency of terms in the corpus, probabilities can be estimated as:

$$
P(t_i, t_j) = \frac{f(t_i, t_j)}{\sum\limits_{(x, y) \in corpus}f(x, y)} 
, P(t_i) = \frac{f(t_i)}{\sum\limits_{t \in corpus}f(t)}
$$

### unigram prebability estimation

In [16]:
U, Un = defaultdict(lambda: 0), 0
for doc in docs:
    for token in tokens(doc):
        U[token] += 1
        Un += 1

In [17]:
p_u = lambda x: U[x] / Un

In [18]:
print(U['school'], Un, p_u('school'))

98 129076 0.0007592426167529208


### bigram probability estimation

In [24]:
print([(a, b, c) for a, b, c in nltk.ngrams('this is a sentence for example'.split(), 3)])

[('this', 'is', 'a'), ('is', 'a', 'sentence'), ('a', 'sentence', 'for'), ('sentence', 'for', 'example')]


In [25]:
B, Bn = defaultdict(lambda: 0), 0
for doc in docs:
    for a, b in nltk.ngrams(tokens(doc), 2):
        B[(a, b)] += 1
        Bn += 1

In [26]:
p_b = lambda x, y: B[(x, y)] / Bn

In [27]:
print(B[('New', 'York')], Bn, p_b('New', 'York'))

204 125948 0.0016197160733000921


### Pmi

In [28]:
PMI = {}
for (a, b), _ in B.items():
    PMI[(a, b)] = np.log(p_b(a, b) / (p_u(a) * p_u(b)))

In [29]:
for (a, b), p in sorted(PMI.items(), key=lambda x: -x[1])[:10]:
    print(a, b, p, U[a], U[b])

Glaucium stigma 11.792688911965513 1 1
stigma lobe 11.792688911965513 1 1
lactucoide Himalayan 11.792688911965513 1 1
subgroup Rhaetian 11.792688911965513 1 1
belltower campanile 11.792688911965513 1 1
triassic sedimentary 11.792688911965513 1 1
illustrious forebear 11.792688911965513 1 1
Mayora Rengifo 11.792688911965513 1 1
UT Cajamarca 11.792688911965513 1 1
physalaemus maculiventris 11.792688911965513 1 1


### Use a threshold on the miminum number of occurrences requested

In [30]:
PMI, th = {}, 5
for (a, b), _ in B.items():
    if U[a] > th and U[b] > th:
        PMI[(a, b)] = np.log(p_b(a, b) / (p_u(a) * p_u(b)))

In [32]:
for (a, b), p in sorted(PMI.items(), key=lambda x: -x[1])[:20]:
    print(a, b, p, U[a], U[b])

Hong Kong 10.000929442737458 6 6
Franche Comté 10.000929442737458 6 6
Fissurellidae keyhole 9.8467787629102 6 7
Serbian Cyrillic 9.818607885943504 6 6
Northwest Territories 9.713247370285677 8 7
Singles Chart 9.664457206116245 7 6
Pearl Jam 9.664457206116245 7 6
Pyramidellidae pyram 9.595464334629295 8 9
Terebridae auger 9.595464334629295 9 9
Buenos Aires 9.490103818971468 10 10
Plot E 9.441313654802036 6 7
Middleton Prestwich 9.307782262177513 10 6
Bourgogne Franche 9.307782262177513 12 6
sudanese Canadians 9.307782262177513 6 6
Brandy Melville 9.307782262177513 8 6
Copa América 9.287162974974777 7 7
terrestrial pulmonate 9.25899209800808 9 7
Trinidad Tobago 9.227739554503977 13 7
Costa Rica 9.227739554503977 13 9
keyhole limpet 9.227739554503977 7 13


### Embed bigrams in tokenization

In [33]:
def pmi_tokenizer(doc, perc=95):
    bigram = []
    words = []
    tks = tokens(doc)
    th = np.percentile(list(PMI.values()), perc)
    for (a, b) in nltk.ngrams(tks, 2):
        if (a, b) in PMI.keys() and PMI[(a, b)] > th:
            if len(bigram) == 0:
                bigram += [a, b]
            else:
                bigram.append(b)
        else:
            if len(bigram) > 0:
                words.append(" ".join(bigram))
                bigram = []
            else:
                words.append(a)
    if tks[-1] != words[-1].split()[-1]:
        words.append(tks[-1])
    return words

In [34]:
pmi_tokens = lambda doc: pmi_tokenizer(doc, perc=90)

In [36]:
print(tokens(docs[180])[40:50])
print(pmi_tokens(docs[180])[40:50])

['model', 'China', 'Special', 'Administrative', 'Regions', 'SARs', 'Hong', 'Kong', 'Macau', 'like']
['Regions', 'SARs', 'Hong Kong', 'Macau', 'like', 'Basic', 'Law', '기본법', 'Kibonpŏp', 'chinese']


### Term-document matrix

In [37]:
M = pd.DataFrame(TfIdf)
M.fillna(0, inplace=True)

In [38]:
M.shape

(25068, 3128)

In [39]:
M.head()

,0,1,2,3,4,5,6,7,8,9,...,3118,3119,3120,3121,3122,3123,3124,3125,3126,3127
be,0.001920,0.001600,0.001440,0.001680,0.001680,0.001280,0.00144,0.001920,0.001600,0.001536,...,0.001600,0.0016,0.001303,0.00192,0.001920,0.001920,0.001920,0.001440,0.00192,0.001920
South,2.789724,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,...,0.000000,0.0000,0.000000,0.00000,0.000000,2.510752,0.000000,0.000000,0.00000,0.000000
Carolina,4.093879,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,...,0.000000,0.0000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
the,0.180774,0.216929,0.216929,0.216929,0.216929,0.216929,0.00000,0.135581,0.180774,0.216929,...,0.198851,0.0000,0.216929,0.00000,0.135581,0.162697,0.162697,0.162697,0.00000,0.144619
Great,3.517040,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,...,0.000000,0.0000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000


In [40]:
M.T.head()

,be,South,Carolina,the,Great,Falls,a,town,in,Chester,...,Whittaker,Masoala,kona,palm,Ochil,Jamestown,Steuben,Morehouse,Artanovsky,225
0,0.00192,2.789724,4.093879,0.180774,3.51704,3.979138,0.109551,2.282118,0.159613,3.655466,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.00160,0.000000,0.000000,0.216929,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.00144,0.000000,0.000000,0.216929,0.00000,0.000000,0.102704,0.000000,0.134674,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.00168,0.000000,0.000000,0.216929,0.00000,0.000000,0.143785,2.139485,0.179565,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.00168,0.000000,0.000000,0.216929,0.00000,0.000000,0.123244,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Text processing with scikit-learn
A tutorial on scikit-learn text processing is available [here](https://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html)

In [44]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

Scikit-learn text facilities expect to work with text as strings instead of pre-tokenized text. Thus, we create a pseudo-text by exploiting our previous tokenizers and creating pseudo words for bigrams.

In [41]:
pseudo_docs = [" ".join([x.replace(' ', '_') for x in pmi_tokens(d)]) for d in docs]

In [42]:
pseudo_docs[180]

'Sinŭiju Special Administrative Region special administrative region SAR North Korea proclaim 2002 de facto operation 2014 border China establish September 2002 area include part Sinŭiju surround area attempt introduce market economic_directly_govern_case directly_govern Cities special administrative region model China Special Administrative Regions SARs Hong_Kong Macau like Basic Law 기본법 Kibonpŏp chinese dutch businessman Yang Bin appoint governor SPA Presidium 2002 formally assume post arrest_chinese_authority_sentence 18 year prison_tax evasion economic_crime north korean_authority soon announce development sinŭiju SAR continue SAR administration_Commission Foreign Economic Cooperation Promotion plan SAR abandon April 2008 SAR reform_effect widely believe North Korea_abandon_project governor_arrest Julie Sa 沙日香 appoint governor 2004'

In [45]:
V = CountVectorizer()

In [46]:
C = V.fit_transform(pseudo_docs)

In [47]:
C

<3128x25779 sparse matrix of type '<class 'numpy.int64'>'
	with 96325 stored elements in Compressed Sparse Row format>

In [48]:
C.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

### Map matrix columns on words

In [49]:
V.vocabulary_.get('hong_kong')

11514

In [50]:
np.nonzero(C[:,11514].toarray())

(array([ 180, 1418, 1792, 2749]), array([0, 0, 0, 0]))

In [52]:
C[1418, 11514]

1

### TfIdf

In [53]:
tf_idf = TfidfTransformer(use_idf=True)
X = tf_idf.fit_transform(C)

In [54]:
X

<3128x25779 sparse matrix of type '<class 'numpy.float64'>'
	with 96325 stored elements in Compressed Sparse Row format>

In [55]:
X.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [56]:
X[180, 11514]

0.08535316687415362

In [58]:
X[180,:].toarray()

array([[0.        , 0.        , 0.        , ..., 0.        , 0.09586641,
        0.        ]])